# ALVIATUL NAZILA
# 202010370311293
# ANALISIS BIG DATA B/D




In [34]:
import pandas as pd
import numpy as np
! pip install kaggle
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cluster import KMeans
%matplotlib inline
plt.style.use('seaborn-whitegrid')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

<ipython-input-34-976e8e4b6c16>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
url = '/content/drive/MyDrive/trace_2023_01_29_09_37_10.csv'
df = pd.read_csv(url)
df

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,172.217.194.95,10.10.1.198,TCP,96,443 > 59620 [ACK] Seq=1 Ack=1 Win=15371 Len=...
1,2,0.000000,172.217.194.95,10.10.1.198,TCP,96,[TCP Dup ACK 1#1] 443 > 59620 [ACK] Seq=1 Ac...
2,3,0.000000,172.217.194.95,10.10.1.198,TCP,96,[TCP Dup ACK 1#2] 443 > 59620 [ACK] Seq=1 Ac...
3,4,0.000101,172.217.194.95,10.10.1.198,TCP,96,443 > 59620 [ACK] Seq=1 Ack=1401 Win=15371 L...
4,5,0.000101,172.217.194.95,10.10.1.198,TCP,96,[TCP Dup ACK 4#1] 443 > 59620 [ACK] Seq=1 Ac...
...,...,...,...,...,...,...,...
250683,250684,45.598711,142.251.10.95,10.10.1.198,TCP,96,443 > 49476 [ACK] Seq=21904 Ack=181073999 Wi...
250684,250685,45.598712,142.251.10.95,10.10.1.198,TCP,96,[TCP Dup ACK 250684#1] 443 > 49476 [ACK] Seq...
250685,250686,45.598712,142.251.10.95,10.10.1.198,TCP,96,443 > 49476 [ACK] Seq=21904 Ack=181075399 Wi...
250686,250687,45.598712,142.251.10.95,10.10.1.198,TCP,96,[TCP Dup ACK 250686#1] 443 > 49476 [ACK] Seq...


### Processing

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250688 entries, 0 to 250687
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   No.          250688 non-null  int64  
 1   Time         250688 non-null  float64
 2   Source       250688 non-null  object 
 3   Destination  250279 non-null  object 
 4   Protocol     250688 non-null  object 
 5   Length       250688 non-null  int64  
 6   Info         250688 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 13.4+ MB


In [38]:
df.isnull().sum()

No.              0
Time             0
Source           0
Destination    409
Protocol         0
Length           0
Info             0
dtype: int64

In [39]:
# Menghapus baris dengan nilai NaN dalam kolom 'Destination'
df.dropna(subset=['Destination'], inplace=True)

In [40]:
df.duplicated().sum()

0

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250279 entries, 0 to 250687
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   No.          250279 non-null  int64  
 1   Time         250279 non-null  float64
 2   Source       250279 non-null  object 
 3   Destination  250279 non-null  object 
 4   Protocol     250279 non-null  object 
 5   Length       250279 non-null  int64  
 6   Info         250279 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 15.3+ MB


### Regex

In [42]:
def extract_info(info_string):
    port_matches = re.findall(r'\d+', info_string.split()[0])
    src_port, dst_port = port_matches if len(port_matches) == 2 else (None, None)

    ack_type = "TCP Dup ACK" if "TCP Dup ACK" in info_string else "ACK" if "ACK" in info_string else None

    seq_match = re.search(r'Seq=(\d+)', info_string)
    ack_match = re.search(r'Ack=(\d+)', info_string)
    seq_val = int(seq_match.group(1)) if seq_match else None
    ack_val = int(ack_match.group(1)) if ack_match else None

    win_match = re.search(r'Win=(\d+)', info_string)
    len_match = re.search(r'Len=(\d+)', info_string)
    win_val = int(win_match.group(1)) if win_match else None
    len_val = int(len_match.group(1)) if len_match else None

    return src_port, dst_port, ack_type, seq_val, ack_val, win_val, len_val

df['Source Port'], df['Destination Port'], df['ACK Type'], df['Seq Value'], df['Ack Value'], df['Win Value'], df['Len Value'] = zip(*df['Info'].map(extract_info))


In [43]:
def extract_ports(info_string):
    port_matches = re.findall(r'(\d+)\s*>\s*(\d+)', info_string)
    if port_matches:
        return port_matches[0]
    else:
        return (None, None)

df['Source Port'], df['Destination Port'] = zip(*df['Info'].map(extract_ports))

df.head()


,No.,Time,Source,Destination,Protocol,Length,Info,Source Port,Destination Port,ACK Type,Seq Value,Ack Value,Win Value,Len Value
0,1,0.000000,172.217.194.95,10.10.1.198,TCP,96,443 > 59620 [ACK] Seq=1 Ack=1 Win=15371 Len=...,443,59620,ACK,1.0,1.0,15371.0,0.0
1,2,0.000000,172.217.194.95,10.10.1.198,TCP,96,[TCP Dup ACK 1#1] 443 > 59620 [ACK] Seq=1 Ac...,443,59620,TCP Dup ACK,1.0,1.0,15371.0,0.0
2,3,0.000000,172.217.194.95,10.10.1.198,TCP,96,[TCP Dup ACK 1#2] 443 > 59620 [ACK] Seq=1 Ac...,443,59620,TCP Dup ACK,1.0,1.0,15371.0,0.0
3,4,0.000101,172.217.194.95,10.10.1.198,TCP,96,443 > 59620 [ACK] Seq=1 Ack=1401 Win=15371 L...,443,59620,ACK,1.0,1401.0,15371.0,0.0
4,5,0.000101,172.217.194.95,10.10.1.198,TCP,96,[TCP Dup ACK 4#1] 443 > 59620 [ACK] Seq=1 Ac...,443,59620,TCP Dup ACK,1.0,1401.0,15371.0,0.0


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250279 entries, 0 to 250687
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   No.               250279 non-null  int64  
 1   Time              250279 non-null  float64
 2   Source            250279 non-null  object 
 3   Destination       250279 non-null  object 
 4   Protocol          250279 non-null  object 
 5   Length            250279 non-null  int64  
 6   Info              250279 non-null  object 
 7   Source Port       241416 non-null  object 
 8   Destination Port  241416 non-null  object 
 9   ACK Type          241414 non-null  object 
 10  Seq Value         241416 non-null  float64
 11  Ack Value         241411 non-null  float64
 12  Win Value         241416 non-null  float64
 13  Len Value         241416 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 28.6+ MB


In [45]:
df.isnull().sum()

No.                    0
Time                   0
Source                 0
Destination            0
Protocol               0
Length                 0
Info                   0
Source Port         8863
Destination Port    8863
ACK Type            8865
Seq Value           8863
Ack Value           8868
Win Value           8863
Len Value           8863
dtype: int64

In [46]:
# Mengisi nilai kosong dengan mean pada kolom 'Seq Value'
mean_seq_value = df['Seq Value'].mean()
df['Seq Value'].fillna(mean_seq_value, inplace=True)

# Mengisi nilai kosong dengan mean pada kolom 'Ack Value'
mean_ack_value = df['Ack Value'].mean()
df['Ack Value'].fillna(mean_ack_value, inplace=True)

# Mengisi nilai kosong dengan mean pada kolom 'Win Value'
mean_win_value = df['Win Value'].mean()
df['Win Value'].fillna(mean_win_value, inplace=True)

# Mengisi nilai kosong dengan mean pada kolom 'Len Value'
mean_len_value = df['Len Value'].mean()
df['Len Value'].fillna(mean_len_value, inplace=True)


In [47]:
# Menghapus baris dengan nilai NaN dalam kolom 'Destination'
df.dropna(subset=['Source Port'], inplace=True)
df.dropna(subset=['Destination Port'], inplace=True)
df.dropna(subset=['ACK Type'], inplace=True)

In [48]:
df.isnull().sum()

No.                 0
Time                0
Source              0
Destination         0
Protocol            0
Length              0
Info                0
Source Port         0
Destination Port    0
ACK Type            0
Seq Value           0
Ack Value           0
Win Value           0
Len Value           0
dtype: int64

### Model

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241412 entries, 0 to 250686
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   No.               241412 non-null  int64  
 1   Time              241412 non-null  float64
 2   Source            241412 non-null  object 
 3   Destination       241412 non-null  object 
 4   Protocol          241412 non-null  object 
 5   Length            241412 non-null  int64  
 6   Info              241412 non-null  object 
 7   Source Port       241412 non-null  object 
 8   Destination Port  241412 non-null  object 
 9   ACK Type          241412 non-null  object 
 10  Seq Value         241412 non-null  float64
 11  Ack Value         241412 non-null  float64
 12  Win Value         241412 non-null  float64
 13  Len Value         241412 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 27.6+ MB


In [50]:
X = df[['Source Port', 'Destination Port', 'Seq Value', 'Ack Value', 'Win Value', 'Len Value']]
y = df['ACK Type']

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [53]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [54]:
y_pred = rf_model.predict(X_test)

In [55]:
# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6577884555640702
